# Kinetic and Fuel Consumption Model

Author: Hraklis Papageorgiou</br>
University of the Witwatersrand</br>
Student: 2137492

#### Reference
Parts of this code is from and adapted from the research paper mentioned below.</br>
C. Hull, J. Giliomee, K. A. Collett, M. D. M. and M. Booysen, “Using high resolution GPS data to plan the electrification of paratransit: A case study in South Africa,” SSRN Electronic Journal, 2022. </br>
Available here:</br>
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4149228</br>
Github:</br>
https://github.com/ChullEPG/Bumpy-Ride</br>
Data:</br>
https://doi.org/10.17632/xt69cnwh56.1


### Packages

In [911]:
import pandas as pd
import os
import numpy as np
from numpy import inf
import datetime
from datetime import datetime, timedelta
from functools import total_ordering
import descartes
import geopandas as gpd
import geopy.distance
from pathlib import Path

### Constants

In [912]:
GRAVITY = 9.80665 # m/s^2
R = 287.05 # J/(kg*K)
KELVIN = 273.15
RHO_CONST = 1.184

# Rolling resistance:
# Assuming pneumatic tyre on a tarmacadam (tarmac) surface
# Car
C_RR_CAR = 0.0225
# Truck
C_RR_TRUCK = 0.01

# Powertrain efficiency

PE_PETROL = 0.202 # Estimated Powertrain efficiency of a petrol vehicle
PE_DIESEL = 0.228 # Estimated Powertrain efficiency of a petrol vehicle
PE_EV = 0.83 # Estimated Powertrain efficiency of an electric vehicle

# Regenerative Braking 

RGN_EV = 0.65
RGN_NGV = 0


# Idle Energy 
P0 = 100



# Kinetic Model
### Vehcile Parameters
csv containing mass, drag coeff and surface area of vehicles by make and model

In [913]:
vehicle_params = pd.read_csv('Vehicles_Constants.csv')
vehicle_params.head(6)

,Make,Model,Curb Mass (kg),Drag Coefficient (C_d),Frontal Surface Area (m2),Displacement (cc),Fuel Type,RPM at Max Torque (rpm)
0,VW,Golf GTI,1644.272,0.355,1.910,1798,Petrol,2000
1,Toyota,HiAce,1800.000,0.360,3.591,2494,Diesel,2000
2,Honda,Accord,1635.000,0.320,2.147,3471,Petrol,4500
3,Honda,Civic 1.2,1530.874,0.380,1.720,1799,Petrol,4250
4,Audi,A3,1644.272,0.310,2.080,1395,Petrol,2000
5,Toyota,Carolla,1530.874,0.290,2.090,1794,Petrol,6750


### Average Vehicle
csv with average mass and drag coeff by vehicle type. </br>
For vehicles that are not available in the data</br>
Frontal surface area can be obtained from a vehicle's heigth and width, (must be provided)

In [914]:
vehicle_avgs = pd.read_csv('vehicle_Averages.csv')
vehicle_avgs.head()

,Type,Curb Mass (kg),Drag Coefficient (C_d),Frontal Surface Area (m2),Displacement (cc)
0,Saloon,1500,0.310,2.2600,2000
1,Sports,1500,0.340,1.8593,3000
2,Light Van,2000,0.425,3.2300,3000
3,Bus,12000,0.600,8.2580,5000
4,Articulated truck,13000,0.675,8.5000,5300


In [915]:
# Return crub mass, c_d and frontal area from make and model of a vehicle
def get_vehicle(make, model):
    vehicle_params = pd.read_csv('Vehicles_Constants.csv')
    try:
        mass_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'Curb Mass (kg)'].iloc[0]
        cd_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'Drag Coefficient (C_d)'].iloc[0]
        area_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'Frontal Surface Area (m2)'].iloc[0]
        w_max_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'RPM at Max Torque (rpm)'].iloc[0]
        d_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'Displacement (cc)'].iloc[0]
        fuel_p = vehicle_params.loc[(vehicle_params['Make'] == make) & (vehicle_params['Model'] == model), 'Fuel Type'].iloc[0]
    except:
        print('Error: Vehicle not found')
        return -1, -1, -1, -1, -1, -1
    
    return mass_p, cd_p, area_p, w_max_p, d_p, fuel_p

In [916]:
mass, cd, area, w_max, d, fuel = get_vehicle('VW', 'Golf GTI')

print(mass)
print(cd)
print(area)
print(w_max)
print(d)
print(fuel)

1644.272
0.355
1.91
2000
1798
Petrol


### File Read

In [917]:
def read_file(filename):
    journey = pd.read_csv(filename)
    return journey

In [918]:
filename = 'Example Input.csv'

journey = read_file(filename)
journey.head(1)

,Date,Time,Latitude,Longitude,Altitude,Speed
0,12/04/2022,07:20:20,-33.89134,18.95951,235.9,1.09


### File Cleaning
• Combine Date and Time </br>
• remove Duplicate data for the same time instance</br>
• Convert speed to velocity in m/s</br>
• Combine coordinatees for geopandas

In [919]:
def clean_data():
    # join date & time
    # Better to do this incase trip is over multiple days, such as over midnight
    journey['DateTime'] = pd.to_datetime(journey['Date'] + journey['Time'], format = '%m/%d/%Y%H:%M:%S')

    # Remove duplicate time samples
    journey.drop_duplicates(subset=['DateTime'], keep="last", inplace=True)

     # Convert from km/h to m/s 
    journey['Velocity'] = np.where(journey['Speed'] >= 0.5, journey['Speed']/3.6, 0)

    # Formats as coordinates
    journey['Coordinates'] = list(zip(journey.Latitude, journey.Longitude)) 

    

In [920]:
clean_data()
journey.head(1)

,Date,Time,Latitude,Longitude,Altitude,Speed,DateTime,Velocity,Coordinates
0,12/04/2022,07:20:20,-33.89134,18.95951,235.9,1.09,2022-12-04 07:20:20,0.302778,"(-33.89134, 18.95951)"


### Slope

In [921]:
''' 
Inputs: dataframe with vehicle journey data

Calculates distance between each successive pair of lat/lon coordinates
(accounts for elevation difference)

Calculates slope angle faced by vehicle at each timestamp

'''
def getDistSlope():
    Distance = np.zeros(len(journey))
    Slope = np.zeros(len(journey))
    l_route = journey.shape[0]

    for i in range(0,l_route-1): 
      #  elev_change = journey['ElevChange'][i]
        elev_change = journey['Altitude'][i + 1] - journey['Altitude'][i]
        if abs(elev_change) < 0.2:
            elev_change = 0
        
        dist_lateral = geopy.distance.geodesic(journey.Coordinates.iloc[i],  # (m) Lateral distance between two lat/lon coord pairs
                                       journey.Coordinates.iloc[i+1]).m
        dist_3d = np.sqrt(dist_lateral**2 + elev_change**2)  # (m) 3D Distance using pythogoras to account for elevation change
        if i == 0:
            Distance[i] = 0
        else:
            Distance[i] = dist_3d
        if Distance[i] != 0 and elev_change != 0:
            Slope[i] = np.arcsin(elev_change/dist_3d)
    journey['Displacement_m'] = list(Distance) # Used for computing kWh/km 
    journey['slope_rad'] = list(Slope) 

In [922]:
getDistSlope()
journey.head(1)

,Date,Time,Latitude,Longitude,Altitude,Speed,DateTime,Velocity,Coordinates,Displacement_m,slope_rad
0,12/04/2022,07:20:20,-33.89134,18.95951,235.9,1.09,2022-12-04 07:20:20,0.302778,"(-33.89134, 18.95951)",0.0,0.0


### Format data and identify features required for kinetic model

• Calculate Change in Elevation</br>
• Calculate Change in Time</br>
• Calculate Change in Velocity </br>
• Calculate Change in Acceleration </br>

In [923]:
def format_data():

    # Change in Time
    journey['DeltaT'] = journey['DateTime'].shift(-1) - journey['DateTime']
    journey.DeltaT = journey['DeltaT']/ np.timedelta64(1, 's') # Convert from timedelta to float

    # Change in Elevation
    journey['ElevChange'] = np.where(abs(journey['Altitude'].shift(-1) - journey['Altitude']) >= 0.2, journey['Altitude'].shift(-1) - journey['Altitude'], 0)

    # Change in Velocity
    journey['DeltaV'] = journey['Velocity'].shift(-1) - journey['Velocity']
    
    # Change in Acceleration
    journey['Acceleration'] = np.where(journey['DeltaT'] > 0, journey['DeltaV']/journey['DeltaT'], 0)

In [924]:
format_data()
journey.head(1)

,Date,Time,Latitude,Longitude,Altitude,Speed,DateTime,Velocity,Coordinates,Displacement_m,slope_rad,DeltaT,ElevChange,DeltaV,Acceleration
0,12/04/2022,07:20:20,-33.89134,18.95951,235.9,1.09,2022-12-04 07:20:20,0.302778,"(-33.89134, 18.95951)",0.0,0.0,2.0,0.0,0.0,0.0


### Environment Forces
• Aerodynamic Drag F_AD </br>
• Slope Derag F_RS or F_SD </br>
• Rolling Resstance F_RR

In [925]:
# Frictional force impeding the motion of a tire rolling on a surface
# F_RR = c_rr * F_N

def rolling_resistance(mass, c_rr, slope, velocity):
    rr = 0
    if velocity > 0.3:
        f_normal = - (mass * GRAVITY * np.cos(slope))
        rr = c_rr * f_normal
    return rr

In [926]:
# Frictional force due to the surrounding air
# F_AD = c_d * rho*v^2 * 1/2*A
# rho = P/(R*T)

def aerodynamic_drag(c_d, velocity, surface_area, rho):
    ad =  c_d * rho * velocity**2 * 0.5
    return -ad


In [927]:
# pressure in hPa & temperature in celsius
def air_density(pressure, temperature):
    temperature += KELVIN
    pressure *= 100
    rho = pressure / (temperature * R)
    return rho

In [928]:
# Positive or negative force due to the slope (component of vehicle weight due to the slope)

def slope_drag(mass, slope):
    sd = mass * GRAVITY * np.sin(slope)
    return -sd

In [929]:
print("Using Average Pressure and temperature: ", air_density(1013.25, 25))
print("Using 1000hPa and 0C", air_density(1000, 40))

Using Average Pressure and temperature:  1.1839251532625141
Using 1000hPa and 0C 1.1124744176078223


## Trip

In [930]:
class Drivecycle: 
    """
    Inputs: dataframe with journey info
    """
    def __init__(self, journey): 
        self.displacement = journey.Displacement_m # m
        self.velocity = journey.Velocity # m/s
        self.acceleration = journey.Acceleration #m/s^2
        self.slope = journey.slope_rad # rad
        self.dt = journey.DeltaT # Time elapsed between each timestamp
        self.dv = journey.DeltaV
        
        

## Vehicle

ErP = Propulsion Energy</br>
ErB = REgenerative Breaking

In [931]:
class Vehicle:
    """
    Inputs: Physical parameters of vehicle for modeling energy consumption
    Returns an array of vehicle energy consumption at each timestamp
    
    Default assumptions from "A bumpy ride to electrification":
    mass = 3900 kg
    cd = 0.36
    crr = 0.02
    A = 4 m^2
    eff = 0.90
    rgbeff = 0.65 # breaking Efficiency
    p0 = 100 W
    """
    
    def __init__(self, mass = 3900, cd = 0.36, crr = 0.02, A = 4,
                 eff = 0.2, rgbeff = 0.65, p0 = 100):

        # Vehicle physical parameters
        self.mass = mass # (kg) Total vehicle weight including payload
        self.crr = crr # Rolling resistance coeff
        self.cd = cd  # Air drag coeff
        self.A = A # (m^2) Approximation of vehicle frontal area 
        self.eff = eff # Powertrain efficiency
        self.rgbeff = rgbeff # Recuperative braking efficiency
        self.p0 = p0 # (W) Constant power loss (to power auxiliary vehicle systems)

        
    # Computes energy expenditure from a Drivecycle object
    def getEnergyExpenditure(self,cycle):

        v = cycle.velocity # (m/s) 
        s = cycle.slope # (rad)
        a = cycle.acceleration # (m/s^2)
        dt = cycle.dt # (s)
        d = cycle.displacement # (m)
        dv = cycle.dv # (m/s)
            
            
        Er = [] # Total energy consumption (J), from battery if positive, to battery if negative (RGbrake)

        
                
        for slope,vel,acc,delta_t, delta_v in zip(s,v,a,dt,dv):
            
            force, frr, fad, fsd = 0,0,0,0
            
            if vel != 0: #No drag if velocity = 0
                frr = rolling_resistance(self.mass,self.crr, slope, vel) # (N)
                fad = aerodynamic_drag(self.cd, vel, self.A, RHO_CONST) # (N)
                fsd = slope_drag(self.mass, slope) # (N)
                
            force = frr + fad + fsd # (N)
            
            exp_speed_delta = force * delta_t / self.mass # (N-s/kg)
            
            unexp_speed_delta = delta_v - exp_speed_delta # (m/s)
            
            try:
                prop_brake_force = self.mass * unexp_speed_delta / delta_t # (kg * m/s)/(s) = (N)
            
                kinetic_power = prop_brake_force * vel # (N)*(m/s) = (W)
                
                propultion_work = kinetic_power * delta_t  # (W)*(s) = (J) 
                
            except ZeroDivisionError:
                prop_brake_force = 0
                kinetic_power = 0
                propultion_work = 0

            Er.append(propultion_work) # (J)

        ErP = [0.0]*len(Er)
        ErB = [0.0]*len(Er)

        for i in range(len(Er)):            
            if Er[i] > 0: # Propulsion Energy 
                ErP[i] = Er[i]/self.eff # (J)
                
            elif Er[i] < 0: # Recuperated Energy (from regen braking system)
                ErB[i]= Er[i]*self.rgbeff # (J)
    
    
    
        return ErP,ErB 

In [932]:
def total_mass(curb, load):
    return curb + load

In [933]:
journey.head(1)

,Date,Time,Latitude,Longitude,Altitude,Speed,DateTime,Velocity,Coordinates,Displacement_m,slope_rad,DeltaT,ElevChange,DeltaV,Acceleration
0,12/04/2022,07:20:20,-33.89134,18.95951,235.9,1.09,2022-12-04 07:20:20,0.302778,"(-33.89134, 18.95951)",0.0,0.0,2.0,0.0,0.0,0.0


## Implementation

In [934]:
minibus = Vehicle(mass = total_mass(mass,800), cd = cd, crr = C_RR_CAR, A = area,
                 eff = PE_DIESEL, rgbeff = RGN_NGV, p0 = P0)

In [935]:
drvcycle = Drivecycle(journey)

In [936]:
p0 = P0

journey['Propultion Work (J)'],journey['Braking Work (J)'] = minibus.getEnergyExpenditure(drvcycle)
journey['Offload Work (J)'] = journey['DeltaT'] * p0 # (J)

journey['Energy Consumption (kWh)'] = (journey['Propultion Work (J)'] + journey['Braking Work (J)'] + journey['Offload Work (J)'])/3.6e6 # (kWh)                
                
#Rename columns for clarity
journey['Displacement (m)'] = journey['Displacement_m']
journey['Slope Angle (rad)'] = journey['slope_rad']

In [937]:
ERp, ERb = minibus.getEnergyExpenditure(drvcycle)

In [938]:
total_kwh = journey['Energy Consumption (kWh)'].sum()

print(total_kwh, 'kWh')

total_dist = journey['Displacement (m)'].sum()/1000
print(total_dist, 'km')

kwh_km = total_kwh/total_dist
print(kwh_km, 'kWh/km')

19.85165470391748 kWh
15.419161469694181 km
1.2874665553593954 kWh/km


# Fuel Consumption Model

In [939]:
# Return crub mass, c_d and frontal area from make and model of a vehicle
def get_epa(make, model):
    vehicle_params = pd.read_csv('vehicles.csv')
    try:
        fe_city_p = vehicle_params.loc[(vehicle_params['make'] == make) & (vehicle_params['model'] == model), 'city08'].iloc[0]
        fe_hwy_p = vehicle_params.loc[(vehicle_params['make'] == make) & (vehicle_params['model'] == model), 'highway08'].iloc[0]
    except:
        print('Error: Vehicle not found')
        return -1, -1
    
    return fe_city_p, fe_hwy_p

### Constants

In [940]:
# Following are general averages

# Density
density_p = 0.75 #kg/litre
density_d = 0.85 #kg/litre

# Air fuel ratio
afr_p = 14.7 #1:14.7 Air
afr_d = 18 #14.5 #1:14.5 Air * Varies a large amount depending on speed

# Fuel efficiency
eff_p = 9.61 #kWh
eff_d = 10.96 #kWh

# CO2 Emissions
emis_p = 2.30531 #kg of CO2
emis_d = 2.68779 #kg of CO2


C_C = 97956215.04 # Conversion constant
V_F = 0.85 # volumetric efficiency of standar engine




In [941]:
def get_air_intake(d, w):
    air_intake = (w*d*V_F)/(C_C)
    return air_intake

For actual implementation, Vehicle engine speed and/or MAF will be avialable at each time step so a calcualtion will be done for each instance of time. Instead of the single calculation done here

In [942]:
def get_fuel_flow_rate(air_intake, afr, fuel_type):
    fuel_flow_rate = air_intake/afr
    fuel_flow_rate *= fuel_type
    return fuel_flow_rate

In [943]:
fuel_flow_rate = get_fuel_flow_rate(get_air_intake(d, w_max), afr_d, density_d)

In [944]:
total_seconds = journey['DeltaT'].sum()
fuel_consumption = total_seconds* fuel_flow_rate
print(fuel_consumption, 'litres of fuel')


2.451920334584985 litres of fuel


In [945]:
km_l = total_dist/fuel_consumption
print(km_l, 'km/l')
l_100 = 100/km_l
print(l_100, 'l/100km')

6.288606221092435 km/l
15.901774810544323 l/100km


In [946]:
emissions = fuel_consumption * emis_d
print(emissions, 'kg of CO2')

6.590246956094177 kg of CO2
